# 임베딩 생성 및 벡터 저장 실습 (OpenAI 임베딩 모델 사용)

- 사용 모델: text-embedding-3-small (OpenAI)
    - OpenAI에서 제공하는 최신 소형 임베딩 모델
    - 빠르고 저렴한 가격 ($0.00002 / 1K tokens)
    - 참고 링크: https://platform.openai.com/docs/guides/embeddings
    - ⚠️ OpenAI API 키가 필수로 필요합니다. (환경변수 또는 직접 입력)
- 실습 목표
    - OpenAI의 text-embedding-3-small 모델 이해하기
    - 로컬이 아닌 API 기반 임베딩 생성 방식 실습
    - 동일한 헌법 텍스트를 벡터화하고, FAISS 인덱스로 저장
    - 앞선 오픈소스 모델들과 비교해보기

# 1. 필수 라이브러리 설치 및 로드

In [1]:
# !pip install openai langchain faiss-cpu python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 환경변수 로드
load_dotenv()

True

# 2. OpenAI 임베딩 모델 설명
- OpenAI의 `text-embedding-3-small`은 OpenAI가 제공하는 최신 임베딩 모델입니다. 
- ChatGPT 및 GPT-4 모델들과 동일한 벡터 공간을 기반으로 작동하기 때문에 RAG 파이프라인에서의 연동성과 응답 품질이 높습니다.
- 벡터 품질이 우수하여 다양한 자연어 질의에 대해 정확한 유사도 계산이 가능합니다.
- 과금이 발생하는 유료 모델입니다. (2025년 5월 기준: $0.00002 / 1K tokens)

# 3. 텍스트 로드 및 문단 단위 분할

In [3]:
file_path = "kr_constitution_cleaned.txt"

with open(file_path, "r", encoding="utf-8") as f:
    full_text = f.read()

In [4]:
# 문단 단위 분할
chunks = full_text.split("\n")
chunks = [c.strip() for c in chunks if len(c.strip()) > 30]

print("총 문단 수:", len(chunks))
print("샘플 문단:", chunks[0])

총 문단 수: 261
샘플 문단: [시행 1988. 2. 25.] [헌법 제10호, 1987. 10. 29., 전부개정]


# 5. 임베딩 모델 로드 (OpenAI)

In [5]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 6. 텍스트 벡터화 및 FAISS 인덱스 생성

In [6]:
faiss_index = FAISS.from_texts(chunks, embedding)
faiss_index.save_local("embeddings/faiss_index_openai")

print("\n✅ FAISS 인덱스 저장 완료")


✅ FAISS 인덱스 저장 완료


# 7. 벡터 검색 테스트

In [7]:
retriever = faiss_index.as_retriever()
retriever.search_kwargs["k"] = 10

query = "국회의 임기는 몇 년인가요?"
results = retriever.invoke(query)

print("\n🔍 질문:", query)
for i, doc in enumerate(results):
    print(f"[{i+1}] {doc.page_content.strip()[:100]}...")


🔍 질문: 국회의 임기는 몇 년인가요?
[1] ②국회의원의 수는 법률로 정하되, 200인 이상으로 한다....
[2] 제130조 ①국회는 헌법개정안이 공고된 날로부터 60일 이내에 의결하여야 하며, 국회의 의결은 재적의원 3분의 2 이상의 찬성을 얻어야 한다....
[3] 제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원 4분의 1 이상의 요구에 의하여 집회된다....
[4] ②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다....
[5] ②이 헌법공포 당시의 국회의원의 임기는 제1항에 의한 국회의 최초의 집회일 전일까지로 한다....
[6] 제3조 ①이 헌법에 의한 최초의 국회의원선거는 이 헌법공포일로부터 6월 이내에 실시하며, 이 헌법에 의하여 선출된 최초의 국회의원의 임기는 국회의원선거후 이 헌법에 의한 국회의 최...
[7] ②원장은 국회의 동의를 얻어 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임할 수 있다....
[8] ②국회의원이 회기 전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면 회기 중 석방된다....
[9] ②법률안에 이의가 있을 때에는 대통령은 제1항의 기간내에 이의서를 붙여 국회로 환부하고, 그 재의를 요구할 수 있다. 국회의 폐회 중에도 또한 같다....
[10] 제61조 ①국회는 국정을 감사하거나 특정한 국정사안에 대하여 조사할 수 있으며, 이에 필요한 서류의 제출 또는 증인의 출석과 증언이나 의견의 진술을 요구할 수 있다....
